In [1]:
import pandas as pd
import ray

In [ ]:
ray.init()

In [6]:
import asd

# The data

In [7]:
data2020 = pd.read_csv("20220727_covid_159rows_52cols_2020.csv")
#data2020["year"] = 2020
data2021 = pd.read_csv("20220727_covid_159rows_52cols_2021.csv")
#data2021["year"] = 2021

In [8]:
data2020.head()

,location,new_cases_per_million,new_deaths_per_million,stringency_index_cov,population_density_cov,population_cov,median_age_cov,aged_65_older_cov,aged_70_older_cov,gdp_per_capita_cov,...,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease,Total_death_ratio,air_polution,Obesity_rate
0,Afghanistan,1313.661,54.938,47.626442,54.422,39835428.0,18.6,2.581,1.337,1803.987,...,0.015165,0.019052,0.010215,0.018640,0.005219,0.004775,0.001507,0.585498,52.99812,5.5
1,Albania,20298.412,411.043,64.760836,104.871,2872934.0,38.0,13.188,8.643,11803.431,...,0.011413,0.028272,0.011552,0.018281,0.000035,0.031810,0.008603,0.771315,18.45981,21.7
2,Algeria,2232.574,61.762,70.767717,17.348,44616626.0,29.1,6.211,3.857,13913.839,...,0.019421,0.017827,0.009579,0.015151,0.000613,0.012335,0.003038,0.455347,32.91557,27.4
3,Angola,517.274,11.899,73.326934,23.890,33933611.0,16.8,2.405,1.362,5819.495,...,0.007997,0.012769,0.018390,0.029117,0.000798,0.003710,0.000867,0.562613,28.02152,8.2
4,Argentina,35642.693,948.224,71.530603,16.177,45605823.0,31.9,11.198,7.441,18933.907,...,0.035312,0.043615,0.019217,0.040107,0.000081,0.027152,0.006860,0.759791,13.84671,28.3


# Predictability Run

Let's specify some columns and targets:

In [9]:
col_set = ["new_cases_per_million", "new_deaths_per_million",
           'Area_km2', "Parkinson's disease", 'Obesity_rate', "Drug use disorders", "Alzheimer's disease and other dementias",
           "Cardiovascular diseases", "Protein-energy malnutrition", "median_age_cov", "aged_70_older_cov", "Density_km2",
           'aged_65_older_cov', 'air_polution', 'stringency_index_cov', 'life_expectancy_cov', 'Diabetes mellitus']
targets=["new_cases_per_million", "new_deaths_per_million"]
primkey_cols=["location", "Year_x"]

and get the corresponding number of combinations

In [10]:
asd.get_column_combinations(all_cols=col_set,inputs=3,outputs=1,targets=targets, amount_only=True)

910

We can scan these combinations with ``method="kNN"``:

In [ ]:
metrics_dict, datas_dict = asd.run_predictability(data=data2020,
                                                  input_cols=3,
                                                  output_cols=1,
                                                  col_set=col_set,
                                                  targets=targets,
                                                  primkey_cols=primkey_cols,
                                                  method="kNN",
                                                  random_state_split=None
                                                  )

Let's have a look at the results:

In [14]:
metrics_df = pd.DataFrame.from_dict(metrics_dict).transpose().sort_values(by="kNN r2", ascending=False)

In [16]:
metrics_df.head(5)

kNN r2  \
median_age_cov                          Diabetes mellitus           Alzheimer's disease and other dementias new_deaths_per_million  0.671875   
Alzheimer's disease and other dementias Obesity_rate                Parkinson's disease                     new_cases_per_million   0.663137   
median_age_cov                          life_expectancy_cov         Obesity_rate                            new_cases_per_million   0.640734   
                                        Density_km2                 stringency_index_cov                    new_deaths_per_million  0.629346   
Drug use disorders                      Protein-energy malnutrition Parkinson's disease                     new_deaths_per_million  0.612063   

                                                                                                                                   linear r2  \
median_age_cov                          Diabetes mellitus           Alzheimer's disease and other dementias new_deaths_per_million  0.586284   
Alzheimer's disease and other dementias Obesity_rate                Parkinson's disease                     new_cases_per_million   0.502738   
median_age_cov                          life_expectancy_cov         Obesity_rate                            new_cases_per_million   0.534042   
                                        Density_km2                 stringency_index_cov                    new_deaths_per_million  0.510133   
Drug use disorders                      Protein-energy malnutrition Parkinson's disease                     new_deaths_per_million  0.494269   

                                                                                                                                   pow. law r2  \
median_age_cov                          Diabetes mellitus           Alzheimer's disease and other dementias new_deaths_per_million        None   
Alzheimer's disease and other dementias Obesity_rate                Parkinson's disease                     new_cases_per_million     0.300214   
median_age_cov                          life_expectancy_cov         Obesity_rate                            new_cases_per_million      0.25993   
                                        Density_km2                 stringency_index_cov                    new_deaths_per_million         NaN   
Drug use disorders                      Protein-energy malnutrition Parkinson's disease                     new_deaths_per_million         NaN   

                                                                                                                                     mean r2  \
median_age_cov                          Diabetes mellitus           Alzheimer's disease and other dementias new_deaths_per_million -0.025568   
Alzheimer's disease and other dementias Obesity_rate                Parkinson's disease                     new_cases_per_million  -0.001246   
median_age_cov                          life_expectancy_cov         Obesity_rate                            new_cases_per_million  -0.009204   
                                        Density_km2                 stringency_index_cov                    new_deaths_per_million -0.002709   
Drug use disorders                      Protein-energy malnutrition Parkinson's disease                     new_deaths_per_million -0.016633   

                                                                                                                                        kNN RMSE  \
median_age_cov                          Diabetes mellitus           Alzheimer's disease and other dementias new_deaths_per_million    188.904554   
Alzheimer's disease and other dementias Obesity_rate                Parkinson's disease                     new_cases_per_million    9809.455364   
median_age_cov                          life_expectancy_cov         Obesity_rate                            new_cases_per_million   11038.622923   
                                        Density_km2                 st

## Refine predictability

In [19]:
selected_tuples = asd.tuple_selection(metrics_dict,
                                  n_best=3
                                  )

In [22]:
selected_tuples

[('median_age_cov',
  'Diabetes mellitus',
  "Alzheimer's disease and other dementias",
  'new_deaths_per_million'),
 ("Alzheimer's disease and other dementias",
  'Obesity_rate',
  "Parkinson's disease",
  'new_cases_per_million'),
 ('median_age_cov',
  'life_expectancy_cov',
  'Obesity_rate',
  'new_cases_per_million')]

Then run the ``refine_predictability`` routine.

In [ ]:
refined_metrics_dict, refined_datas_dict = asd.refine_predictability(best_tuples=selected_tuples,
                                                                     data_dict=datas_dict,
                                                                     time_left_for_this_task=120,
                                                                     n_jobs=-1,
                                                                     use_ray=True
                                                                     )

We may now compare the results:

In [29]:
for key in list(refined_metrics_dict.keys()):
    print(f'{key}: \n \t r2: {round(metrics_dict[key]["kNN r2"],2)}\t\t\t'
          f'RMSE: {round(metrics_dict[key]["kNN RMSE"],2)}\n '
          f'\t ref.: {round(refined_metrics_dict[key]["r2"],2)}\t\t\t'
          f'ref.: {round(refined_metrics_dict[key]["RMSE"],2)}\n')

('median_age_cov', 'Diabetes mellitus', "Alzheimer's disease and other dementias", 'new_deaths_per_million'): 
 	 r2: 0.67			RMSE: 188.9
 	 ref.: 0.61			ref.: 205.06

("Alzheimer's disease and other dementias", 'Obesity_rate', "Parkinson's disease", 'new_cases_per_million'): 
 	 r2: 0.66			RMSE: 9809.46
 	 ref.: 0.41			ref.: 12971.0

('median_age_cov', 'life_expectancy_cov', 'Obesity_rate', 'new_cases_per_million'): 
 	 r2: 0.64			RMSE: 11038.62
 	 ref.: 0.62			ref.: 11342.8



# Plotting

In [31]:
asd.predictability_plot(refined_datas_dict,
                        list(refined_datas_dict.keys())[1],
                        refined_plot=True,
                        initial_datas_dict=datas_dict,
                        plot_along=["init",
                                    "linear",
                                    "pl",
                                    "mean"
                                    ]
                        )